<a href="https://colab.research.google.com/github/LeeHakHyun/DeepLearning/blob/master/colab_githun_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [1] 필요한 library load

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-beta1
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

     |████████████████████████████████| 348.9MB 105kB/s 
     |████████████████████████████████| 3.1MB 50.7MB/s 
     |████████████████████████████████| 501kB 57.1MB/s 


# [2] MNIST 데이터셋 load

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 채널 차원을 추가합니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

11493376/11490434 [==============================] - 0s 0us/step


# [3] Batch 생성

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# [4] keras 모델 생성

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

# [5] loss function과 optimizer 정의

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

# [6] Loss를 계산하고 저장 

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# [7] GradientTape을 이용해 학습 진행

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

# [8] train step 정의 

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

# [9] train 진행

In [16]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.13847769796848297, 정확도: 95.82666778564453, 테스트 손실: 0.06859619170427322, 테스트 정확도: 97.8699951171875
에포크: 2, 손실: 0.08996602892875671, 정확도: 97.27083587646484, 테스트 손실: 0.06318934261798859, 테스트 정확도: 97.99500274658203
에포크: 3, 손실: 0.06645364314317703, 정확도: 97.97999572753906, 테스트 손실: 0.06267097592353821, 테스트 정확도: 98.0633316040039
에포크: 4, 손실: 0.05287032201886177, 정확도: 98.38333129882812, 테스트 손실: 0.06145394593477249, 테스트 정확도: 98.10749816894531
에포크: 5, 손실: 0.044248390942811966, 정확도: 98.63899993896484, 테스트 손실: 0.06251437962055206, 테스트 정확도: 98.14800262451172
